In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### External Imports

In [12]:
import os
import torch
import torchvision as tv
import random
import cv2 as cv
import numpy as np
import typing as t
import pathlib as pb
from ultralytics import YOLO

### Internal Imports

In [13]:
from vistraff.res import Assets
from vistraff.model import FasterRCNNMOD
from vistraff.alg import VehicleOccupancy, VehicleTracking
from vistraff.data import CarTrafficDataset, CarTrafficTaskOneDict, CarTrafficTaskTwoDict

### Environment Setup

In [14]:
PATH_ROOT: pb.Path = pb.Path('.')
PATH_ASSETS: pb.Path = PATH_ROOT / 'assets'
PATH_MASKS: pb.Path = PATH_ASSETS / 'masks'
PATH_MODELS: pb.Path = PATH_ASSETS / 'models'
PATH_DATA: pb.Path = PATH_ROOT / 'data' / 'train'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [15]:
SEED: int = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
np.random.default_rng(SEED)
torch.set_grad_enabled(False)

### Data Loading

In [16]:
dataset: CarTrafficDataset = CarTrafficDataset(PATH_DATA)
assets: Assets = Assets(PATH_ASSETS)

### Task 1

In [17]:
# Create the task solver
vo = VehicleOccupancy(assets, device)

# Solve each example in the task subset
for i in range(len(dataset.task_1)):
    data: CarTrafficTaskOneDict = dataset.task_1[i]
    output: t.List[int] = vo(data)
    output_file: pb.Path = dataset.task_1.path / f"{data['context']:02}_{data['example']}_predicted.txt"
    cv.destroyAllWindows()

    # Write the answers
    with open(output_file, 'w') as out_file:
        out_file.write(str(len(output)) + os.linesep)
        for i, (query, answer) in enumerate(zip(data['query'], output)):
            out_file.write(str(query) + ' ' + str(answer))
            if i < len(output) - 1:
                out_file.write(os.linesep)

### Task 2

In [ ]:
# Iterate through each video in the dataset
for i in range(len(dataset.task_2)):
    # Create task solver for each video
    vt = VehicleTracking(assets, device)

    # Track the vehicle in each frame of the video
    for data in dataset.task_2[i]:
        bbox: t.List[int] = vt(data)




### Task 3

In [7]:
model = YOLO(model='./private/vistraff/first-attempt3/weights/best.pt', task='track')

### Prediction

### Saving and evaluating the results

In [ ]:
d